# 기본 설정

In [2]:
# 설정 ############################
import os
import glob
import shutil
import random
import xml.etree.ElementTree as ET
import subprocess
import cv2
import torch
from ultralytics import YOLO

DATASET_DIR = "C:/data/PKLot"
YOLO_LABELS_DIR = os.path.join(DATASET_DIR, "YOLO_labels")
IMAGES_DIR = os.path.join(DATASET_DIR, "images")
LABELS_DIR = os.path.join(DATASET_DIR, "labels")
YOLO_YAML_PATH = os.path.join(DATASET_DIR, "yolo_dataset.yaml")

print("✅ 설정 완료.")

✅ 설정 완료.


# XML to YOLO 변환

In [ ]:
def convert_xml_to_yolo(xml_path, output_dir):
    tree = ET.parse(xml_path)
    root = tree.getroot()

    image_name = os.path.basename(xml_path).replace(".xml", ".jpg")
    label_path = os.path.join(output_dir, image_name.replace(".jpg", ".txt"))

    with open(label_path, "w") as f:
        for space in root.findall("space"):
            occupied = int(space.get("occupied", 0))  # 0: empty, 1: occupied
            rect = space.find("rotatedRect")

            if rect is None:
                continue

            center = rect.find("center")
            size = rect.find("size")

            try:
                x, y = int(center.get("x")), int(center.get("y"))
                w, h = int(size.get("w")), int(size.get("h"))
            except (TypeError, AttributeError):
                continue

            # YOLO 좌표 변환 (정규화)
            img_width, img_height = 1280, 720  # 기본값 (수정 필요)
            x, y, w, h = x / img_width, y / img_height, w / img_width, h / img_height

            f.write(f"{occupied} {x} {y} {w} {h}\n")

if not os.path.exists(YOLO_LABELS_DIR):
    os.makedirs(YOLO_LABELS_DIR)

xml_files = glob.glob(os.path.join(DATASET_DIR, "**", "*.xml"), recursive=True)

for xml_file in xml_files:
    convert_xml_to_yolo(xml_file, YOLO_LABELS_DIR)

print(f"✅ XML 변환 완료: {len(xml_files)}개 파일 처리됨.")


✅ XML 변환 완료: 12416개 파일 처리됨.


# 데이터셋 정리

In [ ]:
if not os.path.exists(IMAGES_DIR):
    os.makedirs(IMAGES_DIR)

if not os.path.exists(LABELS_DIR):
    os.makedirs(LABELS_DIR)

# 모든 이미지 가져오기
image_files = glob.glob(os.path.join(DATASET_DIR, "**", "*.jpg"), recursive=True)

# 데이터셋 나누기 (train: 80%, val: 10%, test: 10%)
random.shuffle(image_files)
num_train = int(len(image_files) * 0.8)
num_val = int(len(image_files) * 0.1)

train_files = image_files[:num_train]
val_files = image_files[num_train:num_train + num_val]
test_files = image_files[num_train + num_val:]

# 폴더 생성
for folder in ["train", "val", "test"]:
    os.makedirs(os.path.join(IMAGES_DIR, folder), exist_ok=True)
    os.makedirs(os.path.join(LABELS_DIR, folder), exist_ok=True)

def copy_files(file_list, target_img_dir, target_lbl_dir):
    for img_path in file_list:
        img_name = os.path.basename(img_path)
        lbl_name = img_name.replace(".jpg", ".txt")
        
        shutil.copy(img_path, os.path.join(target_img_dir, img_name))
        lbl_path = os.path.join(YOLO_LABELS_DIR, lbl_name)
        
        if os.path.exists(lbl_path):
            shutil.copy(lbl_path, os.path.join(target_lbl_dir, lbl_name))

copy_files(train_files, os.path.join(IMAGES_DIR, "train"), os.path.join(LABELS_DIR, "train"))
copy_files(val_files, os.path.join(IMAGES_DIR, "val"), os.path.join(LABELS_DIR, "val"))
copy_files(test_files, os.path.join(IMAGES_DIR, "test"), os.path.join(LABELS_DIR, "test"))

print("✅ 데이터셋 분할 완료.")

# yolo_dataset.yaml 생성

In [ ]:
yaml_content = f"""path: {DATASET_DIR}
train: images/train
val: images/val
test: images/test
nc: 2
names: ["empty", "occupied"]
"""

with open(YOLO_YAML_PATH, "w") as f:
    f.write(yaml_content)

print("✅ YOLO 데이터셋 YAML 생성 완료.")

# YAML 파일 존재 여부 확인
if not os.path.exists(YOLO_YAML_PATH):
    raise FileNotFoundError(f"❌ {YOLO_YAML_PATH} 파일이 존재하지 않습니다! YAML 생성이 실패했을 가능성이 있습니다.")

✅ YOLO 데이터셋 YAML 생성 완료.


In [2]:
# 모델 경로 설정 (YOLOv11의 사전 학습된 가중치 사용)
model_path = "yolo11n.pt"  # 또는 "yolov5s.pt" 등 사용

# 데이터셋 YAML 경로
DATASET_YAML_PATH = "C:/data/PKLot/yolo_dataset.yaml"

# 모델 불러오기
model = YOLO(model_path)  # ✅ 절대 경로를 사용할 수도 있음

# ✅ YOLO 학습 실행 (실시간 로그 출력)
model.train(data=DATASET_YAML_PATH,  # ✅ 데이터셋 경로
            epochs=5,    # ✅ 학습 epoch 설정
            batch=8,     # ✅ 배치 크기 설정
            imgsz=640)  # ✅ 이미지 크기 설정

# ✅ 학습된 모델 저장 (최적 모델 저장)
MODEL_SAVE_PATH = "C:/data/PKLot/best.pt"
model.save(MODEL_SAVE_PATH)

print(f"✅ YOLOv11 학습 완료! 모델이 {MODEL_SAVE_PATH}에 저장되었습니다.")


New https://pypi.org/project/ultralytics/8.3.79 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.78  Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolo11n.pt, data=C:/data/PKLot/yolo_dataset.yaml, epochs=5, time=None, patience=100, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train8, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=Non

train: Scanning C:\data\PKLot\labels\train.cache... 9933 images, 1 backgrounds, 0 corrupt: 100%|██████████| 9934/9934 [00:00<?, ?it/s]
INFO:albumentations.check_version:A new version of Albumentations is available: 2.0.4 (you have 1.4.10). Upgrade using: pip install --upgrade albumentations


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


val: Scanning C:\data\PKLot\labels\val.cache... 1241 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1241/1241 [00:00<?, ?it/s]


Plotting labels to runs\detect\train8\labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001667, momentum=0.9) with parameter groups 81 weight(decay=0.0), 88 weight(decay=0.0005), 87 bias(decay=0.0)
TensorBoard: model graph visualization added 
Image sizes 640 train, 640 val
Using 8 dataloader workers
Logging results to runs\detect\train8
Starting training for 5 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all       1241      72284      0.971      0.968      0.992      0.841

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


        5/5      4.79G     0.7183     0.4598     0.8275        571        640: 100%|██████████| 1242/1242 [02:26<00:00,  8.46it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 78/78 [00:08<00:00,  8.88it/s]


                   all       1241      72284      0.985      0.985      0.994      0.875

5 epochs completed in 0.222 hours.
Optimizer stripped from runs\detect\train8\weights\last.pt, 5.4MB
Optimizer stripped from runs\detect\train8\weights\best.pt, 5.4MB

Validating runs\detect\train8\weights\best.pt...
Ultralytics 8.3.78  Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 78/78 [00:23<00:00,  3.29it/s]


                   all       1241      72284      0.985      0.985      0.994      0.875
                 empty       1085      39968      0.979      0.996      0.995      0.888
              occupied        956      32316      0.991      0.973      0.993      0.862
Speed: 0.2ms preprocess, 1.2ms inference, 0.0ms loss, 1.0ms postprocess per image
Results saved to runs\detect\train8
✅ YOLOv11 학습 완료! 모델이 C:/data/PKLot/best.pt에 저장되었습니다.


# 성능평가

In [4]:
from ultralytics import YOLO

# ✅ 학습된 모델 불러오기
MODEL_PATH = "C:/data/PKLot/best.pt"
model = YOLO(MODEL_PATH)

# ✅ 성능 평가 수행 (mAP, Precision, Recall 계산)
results = model.val()

# ✅ 평가 결과 출력
print("📊 YOLOv11 성능 평가 결과:")
print(results)

Ultralytics 8.3.78  Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 4070 Laptop GPU, 8188MiB)
YOLO11n summary (fused): 100 layers, 2,582,542 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning C:\data\PKLot\labels\val.cache... 1241 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1241/1241 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 78/78 [00:19<00:00,  4.03it/s]


                   all       1241      72284      0.985      0.985      0.994      0.877
                 empty       1085      39968      0.979      0.996      0.995       0.89
              occupied        956      32316      0.991      0.973      0.993      0.864
Speed: 0.2ms preprocess, 1.8ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\detect\val
📊 YOLOv11 성능 평가 결과:
ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x000002BDBD9BA690>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018

# 이미지 테스트

In [29]:
# ✅ 모델 불러오기
MODEL_PATH = "C:/data/PKLot/best.pt"
# MODEL_PATH = "C:/test/wonjeonghwan/yolo11n.pt"
model = YOLO(MODEL_PATH)

# ✅ 이미지 예측 함수
def predict_image(image_path):
    img = cv2.imread(image_path)  # 이미지 읽기
    results = model(img)  # 모델 예측
    results[0].show()  # 예측 결과 표시
    return results

# ✅ 테스트할 이미지 경로
IMAGE_PATH = "C:/Jeonghwan/data/archive/parking/for_test.jpg"

# ✅ 이미지 예측 실행
print("🔍 이미지 예측 시작...")
predict_image(IMAGE_PATH)


🔍 이미지 예측 시작...

0: 576x640 (no detections), 40.8ms
Speed: 2.4ms preprocess, 40.8ms inference, 0.5ms postprocess per image at shape (1, 3, 576, 640)


[ultralytics.engine.results.Results object with attributes:
 
 boxes: ultralytics.engine.results.Boxes object
 keypoints: None
 masks: None
 names: {0: 'empty', 1: 'occupied'}
 obb: None
 orig_img: array([[[103, 112, 116],
         [110, 119, 123],
         [113, 122, 126],
         ...,
         [131, 147, 163],
         [105, 124, 137],
         [105, 124, 137]],
 
        [[ 83,  92,  96],
         [ 98, 107, 111],
         [109, 118, 122],
         ...,
         [112, 129, 142],
         [107, 126, 139],
         [107, 126, 139]],
 
        [[ 55,  66,  70],
         [ 76,  87,  91],
         [ 99, 110, 114],
         ...,
         [106, 123, 136],
         [106, 123, 136],
         [106, 124, 135]],
 
        ...,
 
        [[ 70,  75,  74],
         [ 74,  79,  78],
         [ 93,  98,  97],
         ...,
         [134, 145, 153],
         [132, 143, 151],
         [132, 143, 151]],
 
        [[ 69,  74,  73],
         [ 73,  78,  77],
         [ 93,  98,  97],
         ...,
    

In [36]:
import cv2
from ultralytics import YOLO

# ✅ 모델 불러오기
MODEL_PATH = "C:/data/PKLot/best.pt"
model = YOLO(MODEL_PATH)

# ✅ 영상 예측 함수
def predict_video(video_path, output_path="C:/data/PKLot/output.mp4"):
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        results = model(frame)  # YOLO 예측 실행
        frame = results[0].plot()  # 예측 결과 반영된 프레임 가져오기
        
        out.write(frame)  # 출력 영상 저장
        
        # ✅ OpenCV GUI 기능 제거
        # cv2.imshow("YOLOv11 Detection", frame)  # 제거
        # cv2.waitKey(1)  # 제거

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# ✅ 테스트할 영상 경로
VIDEO_PATH = "C:/Jeonghwan/data/archive/parking/Top_view.mp4"

# ✅ 영상 예측 실행
print("🎥 영상 예측 시작...")
predict_video(VIDEO_PATH)

print("✅ 영상 예측 완료! 결과가 C:/data/PKLot/output.mp4 에 저장되었습니다.")


🎥 영상 예측 시작...

0: 384x640 2 emptys, 2 occupieds, 9.8ms
Speed: 2.3ms preprocess, 9.8ms inference, 1.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 emptys, 1 occupied, 11.7ms
Speed: 1.8ms preprocess, 11.7ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 emptys, 1 occupied, 9.2ms
Speed: 1.6ms preprocess, 9.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 emptys, 4 occupieds, 8.2ms
Speed: 1.4ms preprocess, 8.2ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 1 empty, 3 occupieds, 11.3ms
Speed: 1.6ms preprocess, 11.3ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 occupieds, 8.5ms
Speed: 1.4ms preprocess, 8.5ms inference, 1.1ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 (no detections), 8.2ms
Speed: 1.4ms preprocess, 8.2ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 occupieds, 7.6ms
Speed: 1.6m

error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window.cpp:1295: error: (-2:Unspecified error) The function is not implemented. Rebuild the library with Windows, GTK+ 2.x or Cocoa support. If you are on Ubuntu or Debian, install libgtk2.0-dev and pkg-config, then re-run cmake or configure script in function 'cvDestroyAllWindows'


In [3]:
dataset_path = "C:/data/"  # 데이터셋 폴더 경로
max_files_per_folder = 4  # 각 폴더에서 출력할 최대 파일 개수

for root, dirs, files in os.walk(dataset_path):
    print(root)
    
    # 파일이 너무 많으면 일부만 출력
    for file in files[:max_files_per_folder]:  # 최대 5개만 출력
        print(f"    ├── {file}")
    
    if len(files) > max_files_per_folder:
        print(f"    ├── ... (총 {len(files)}개 파일 중 {max_files_per_folder}개만 표시됨)")

    print()

C:/data/
    ├── licence
    ├── PklotInfo.pdf
    ├── readme.txt

C:/data/PKLot
    ├── best.pt
    ├── licence
    ├── output.mp4
    ├── PklotInfo.pdf
    ├── ... (총 6개 파일 중 4개만 표시됨)

C:/data/PKLot\images

C:/data/PKLot\images\test
    ├── 2012-09-11_15_36_32.jpg
    ├── 2012-09-11_15_38_53.jpg
    ├── 2012-09-11_15_45_57.jpg
    ├── 2012-09-11_15_55_21.jpg
    ├── ... (총 1243개 파일 중 4개만 표시됨)

C:/data/PKLot\images\train
    ├── 2012-09-11_15_16_58.jpg
    ├── 2012-09-11_15_27_08.jpg
    ├── 2012-09-11_15_29_29.jpg
    ├── 2012-09-11_15_31_50.jpg
    ├── ... (총 9934개 파일 중 4개만 표시됨)

C:/data/PKLot\images\val
    ├── 2012-09-11_15_43_35.jpg
    ├── 2012-09-11_17_49_51.jpg
    ├── 2012-09-11_18_18_05.jpg
    ├── 2012-09-12_06_36_36.jpg
    ├── ... (총 1241개 파일 중 4개만 표시됨)

C:/data/PKLot\labels
    ├── train.cache
    ├── val.cache

C:/data/PKLot\labels\test
    ├── 2012-09-11_15_36_32.txt
    ├── 2012-09-11_15_38_53.txt
    ├── 2012-09-11_15_45_57.txt
    ├── 2012-09-11_15_55_21.txt
    ├──